In [1]:
import os
import json
import requests
import pandas as pd
import numpy as np
import time 
import matplotlib.pyplot as plt

from functions import load_api_key

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
api_key = load_api_key()

# MRT coordinates

In [3]:
line_ls = ['EW', 'NS', 'NE', 'CC', 'DT', 'TE', 'BP', 'SE', 'SW', 'PE', 'PW']

station_ls = []

for line in line_ls:
    temp_ls = [line + str(i) for i in range(1, 40)]
    station_ls = station_ls + temp_ls
station_ls

['EW1',
 'EW2',
 'EW3',
 'EW4',
 'EW5',
 'EW6',
 'EW7',
 'EW8',
 'EW9',
 'EW10',
 'EW11',
 'EW12',
 'EW13',
 'EW14',
 'EW15',
 'EW16',
 'EW17',
 'EW18',
 'EW19',
 'EW20',
 'EW21',
 'EW22',
 'EW23',
 'EW24',
 'EW25',
 'EW26',
 'EW27',
 'EW28',
 'EW29',
 'EW30',
 'EW31',
 'EW32',
 'EW33',
 'EW34',
 'EW35',
 'EW36',
 'EW37',
 'EW38',
 'EW39',
 'NS1',
 'NS2',
 'NS3',
 'NS4',
 'NS5',
 'NS6',
 'NS7',
 'NS8',
 'NS9',
 'NS10',
 'NS11',
 'NS12',
 'NS13',
 'NS14',
 'NS15',
 'NS16',
 'NS17',
 'NS18',
 'NS19',
 'NS20',
 'NS21',
 'NS22',
 'NS23',
 'NS24',
 'NS25',
 'NS26',
 'NS27',
 'NS28',
 'NS29',
 'NS30',
 'NS31',
 'NS32',
 'NS33',
 'NS34',
 'NS35',
 'NS36',
 'NS37',
 'NS38',
 'NS39',
 'NE1',
 'NE2',
 'NE3',
 'NE4',
 'NE5',
 'NE6',
 'NE7',
 'NE8',
 'NE9',
 'NE10',
 'NE11',
 'NE12',
 'NE13',
 'NE14',
 'NE15',
 'NE16',
 'NE17',
 'NE18',
 'NE19',
 'NE20',
 'NE21',
 'NE22',
 'NE23',
 'NE24',
 'NE25',
 'NE26',
 'NE27',
 'NE28',
 'NE29',
 'NE30',
 'NE31',
 'NE32',
 'NE33',
 'NE34',
 'NE35',
 'NE36',
 

In [4]:
station_df = pd.DataFrame(station_ls, columns = ['station_code'])
station_df['station_name'] = None
station_df['postal_code']  = None
station_df['latitude']     = None
station_df['longitude']    = None
station_df

,station_code,station_name,postal_code,latitude,longitude
0,EW1,None,None,None,None
1,EW2,None,None,None,None
2,EW3,None,None,None,None
3,EW4,None,None,None,None
4,EW5,None,None,None,None
...,...,...,...,...,...
424,PW35,None,None,None,None
425,PW36,None,None,None,None
426,PW37,None,None,None,None
427,PW38,None,None,None,None


In [5]:
start = time.time()
for station in station_ls:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={station}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {"Authorization": api_key}
    response = requests.get(url, headers=headers)
    parsed_data = json.loads(response.text)

    for count in range(len(parsed_data['results'])):  
         if ('MRT' in parsed_data['results'][count]['SEARCHVAL']) or ('LRT' in parsed_data['results'][count]['SEARCHVAL']):
             station_df.loc[station_df['station_code'] == station, 'station_name'] = parsed_data['results'][count]['SEARCHVAL']
             station_df.loc[station_df['station_code'] == station, 'postal_code'] = parsed_data['results'][count]['POSTAL']
             station_df.loc[station_df['station_code'] == station, 'latitude'] = parsed_data['results'][count]['LATITUDE']
             station_df.loc[station_df['station_code'] == station, 'longitude'] = parsed_data['results'][count]['LONGITUDE'] 
    
    print(station, start -  time.time())

EW1 -0.4602017402648926
EW2 -0.9135892391204834
EW3 -1.3684751987457275
EW4 -1.83856201171875
EW5 -2.3015639781951904
EW6 -2.7539570331573486
EW7 -3.2142410278320312
EW8 -3.6557209491729736
EW9 -4.094177484512329
EW10 -4.561285734176636
EW11 -5.000656604766846
EW12 -5.433922052383423
EW13 -5.872959852218628
EW14 -6.320564031600952
EW15 -6.760585308074951
EW16 -7.194989442825317
EW17 -7.641293048858643
EW18 -8.085492372512817
EW19 -9.15543508529663
EW20 -9.657351016998291
EW21 -10.11131739616394
EW22 -10.569202423095703
EW23 -10.996354579925537
EW24 -11.461754322052002
EW25 -11.894477605819702
EW26 -12.325553894042969
EW27 -12.760667085647583
EW28 -13.207015752792358
EW29 -13.642367601394653
EW30 -14.08495831489563
EW31 -14.527742624282837
EW32 -14.998775005340576
EW33 -15.43927264213562
EW34 -16.01550531387329
EW35 -16.48339819908142
EW36 -16.937191486358643
EW37 -17.3732168674469
EW38 -17.829914331436157
EW39 -18.281745195388794
NS1 -18.723224639892578
NS2 -19.173104763031006
NS3 -19.


KeyboardInterrupt



In [ ]:
station_df[['station_name', 'postal_code', 'latitude', 'longitude']]

In [ ]:
station_df = station_df.drop_duplicates(subset = ['station_name', 'postal_code', 'latitude', 'longitude'], keep='first')
station_df = station_df.loc[station_df['postal_code'] != 'NIL']
station_df = station_df.dropna()

In [ ]:
station_df